In [22]:
import joblib
import function_library as fl
import importlib
import pandas as pd
import time
importlib.reload(fl)

<module 'function_library' from 'C:\\Users\\leere\\PycharmProjects\\Football_ML3\\function_library.py'>

In [23]:
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\GOALS_LIVE.CSV"


# List of common encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")

col_dict = {
    "country": "Country",
    "league": "League",
    "datameci": "Date",
    "etapa": "Round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "place1t": "Home_team_place_total",
    "place1a": "Home_team_place_home",
    "place2t": "Away_team_place_total",
    "place2d": "Away_team_place_away",
    "customh": "ELO_home",
    "customa": "ELO_away",
    "custom3": "FORM_home",
    "custom4": "FORM_away",
    "home_val": "home_win",
    "home_val_2": "home_win_15",
    "home_val_3": "home_o25",
    "home_val_4": "home_o35",
    "home_val_5": "home_scored",
    "away_val": "away_win",
    "away_val_2": "away_win_15",
    "away_val_3": "away_o25",
    "away_val_4": "away_o35",
    "away_val_5": "away_scored",
    "scor1": "home_goals",
    "scor2": "away_goals",
    "cotao": "o2.5_odds",
}

data = data.rename(columns=col_dict).filter(items=col_dict.values())
# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Order by date
data = data.sort_values(by='Date')

data = data[data["Round"] >= 8]
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

cols_to_drop = ['Date','home_team', 'away_team', 'home_goals', 'away_goals', 'o2.5_odds']

data_ready =data.drop(columns=cols_to_drop)
data_ready

Successfully read the file with encoding: utf-8


,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,ELO_home,ELO_away,FORM_home,...,home_win,home_win_15,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored
0,Ireland,Premier D,8,1,1,4,2,1913.0,1830.0,2.0,...,75.000,50.000,50.000,25.000,2.250,66.667,0.000,33.333,0.000,1.333
1,France,Ligue 2,29,15,15,16,8,1527.0,1445.0,8.0,...,38.462,15.385,69.231,38.462,1.769,38.462,7.692,38.462,23.077,0.846
2,France,Ligue 2,29,3,3,17,11,1643.0,1406.0,4.0,...,71.429,28.571,64.286,42.857,2.071,30.769,0.000,46.154,23.077,0.692
3,Turkey,1. Lig,32,6,1,20,20,1824.0,1273.0,10.0,...,73.333,46.667,66.667,33.333,2.200,0.000,0.000,81.250,43.750,0.250
4,Portugal,Primeira L,28,16,15,9,9,1670.0,1800.0,2.0,...,23.077,0.000,30.769,23.077,0.846,23.077,7.692,69.231,53.846,1.308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Scotland,Premier L,32,12,12,1,1,1688.0,2190.0,3.0,...,26.667,6.667,66.667,20.000,1.067,73.333,60.000,73.333,40.000,2.600
273,Italy,Serie B,32,3,3,18,14,1630.0,1473.0,-3.0,...,60.000,20.000,53.333,26.667,1.800,13.333,13.333,66.667,46.667,1.533
274,Italy,Serie B,32,7,12,1,1,1562.0,1754.0,6.0,...,40.000,20.000,26.667,13.333,1.133,60.000,26.667,53.333,20.000,1.733
275,Portugal,Primeira L,28,3,3,2,3,2046.0,2111.0,-1.0,...,76.923,61.538,46.154,23.077,2.308,61.538,30.769,46.154,23.077,1.692


In [24]:
# Directory where models are saved
models_dir = "ALL_MODELS"

# Group unseen data by League and Sub-League
grouped_data = data_ready.groupby(['Country', 'League'])

# Dictionary to store predictions
predictions = {}

# Iterate through each group in the unseen data
for (country, league), group in grouped_data:
    # Construct the model filename
    model_filename = f"{models_dir}/{country}_{league}_model.pkl"

    try:
        # Load the corresponding model
        saved_data = joblib.load(model_filename)  # Load the entire dictionary
        model = saved_data['model']  # Extract the model
        scaler = saved_data['scaler']  # Extract the scaler
        threshold = saved_data['threshold']  # Extract the threshold

        # Extract features for prediction (exclude grouping columns)
        X = group.drop(columns=['Country', 'League'])

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make probability predictions using the model
        y_proba = model.predict_proba(X_scaled)[:, 1]  # Positive class probabilities

        # Filter predictions based on the threshold
        group['Probability'] = y_proba
        group['Prediction'] = (y_proba >= threshold).astype(int)

        # Keep only rows where Prediction == 1
        filtered_predictions = group[group['Prediction'] == 1]

        # Add `home_team` and `away_team` from the original `data`
        game_info = data.loc[filtered_predictions.index, ['home_team', 'away_team', 'Date']]
        filtered_predictions = pd.concat([game_info, filtered_predictions], axis=1)

        # Add filtered predictions to the dictionary
        predictions[(country, league)] = filtered_predictions
        print(f"Filtered predictions made for {country} - {league}")

    except FileNotFoundError:
        print(f"Model not found for {country} - {league}. Skipping...")

    except Exception as e:
        print(f"Error processing {country} - {league}: {e}")

# Combine all predictions into a single DataFrame
if predictions:
    all_predictions = pd.concat(predictions.values(), ignore_index=True)
    print("\nAll Predictions as a DataFrame:")
    print(all_predictions)
else:
    print("No predictions available.")


Filtered predictions made for Argentina - Primera Divisio
Model not found for Australia - A-League. Skipping...
Filtered predictions made for Austria - Erste Liga
Filtered predictions made for Bulgaria - Premier L
Filtered predictions made for Croatia - Division 1
Filtered predictions made for Czech - Division 1
Filtered predictions made for England - Championship
Filtered predictions made for England - League One
Filtered predictions made for England - League Two
Filtered predictions made for England - Premier L
Filtered predictions made for France - Ligue 1
Filtered predictions made for France - Ligue 2
Filtered predictions made for Germany - 3rd League
Filtered predictions made for Germany - Bundesliga
Filtered predictions made for Germany - Bundesliga 2
Filtered predictions made for Hungary - Division 1
Filtered predictions made for Ireland - Premier D
Filtered predictions made for Italy - Serie A
Filtered predictions made for Italy - Serie B
Filtered predictions made for Japan - J

In [25]:
#all_predictions

In [26]:
all_predictions = fl.team_name_map(all_predictions)
# Get today's date (normalized to midnight so we only compare dates)

In [27]:
today = pd.Timestamp.today().normalize()
# Filter the DataFrame to include only rows with dates greater than or equal to today
all_predictions = all_predictions[all_predictions['Date'] >= today]
all_predictions

,home_team,away_team,Date,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,...,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored,Probability,Prediction
0,Talleres,Gimnasia La Plata,2025-04-05,Argentina,Primera Divisio,12,23,30,21,26,...,40.000,20.000,0.800,0.000,0.000,33.333,0.000,0.167,0.665992,1
1,Boca Juniors,Barracas Central,2025-04-06,Argentina,Primera Divisio,12,3,2,12,10,...,50.000,16.667,1.833,20.000,20.000,20.000,20.000,1.200,0.530197,1
2,Sarmiento de Junin,River Plate,2025-04-06,Argentina,Primera Divisio,12,24,18,7,11,...,20.000,0.000,0.800,20.000,20.000,0.000,0.000,0.600,0.577925,1
3,SV Ried,SKU Amstetten,2025-04-04,Austria,Erste Liga,22,2,3,9,12,...,70.000,40.000,2.400,30.000,10.000,50.000,20.000,1.200,0.656916,1
4,Bregenz,SV Stripfing/Weiden,2025-04-04,Austria,Erste Liga,22,3,2,14,15,...,81.818,54.545,2.273,0.000,0.000,50.000,20.000,0.700,0.641761,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Villarreal,Athletic Bilbao,2025-04-06,Spain,Primera,30,5,13,4,5,...,76.923,53.846,2.231,42.857,7.143,50.000,14.286,1.357,0.554229,1
63,Lausanne,Sion,2025-04-05,Switzerland,Super League,31,7,5,9,11,...,66.667,40.000,1.867,20.000,6.667,53.333,33.333,0.933,0.751082,1
64,Winterthur,FC Zurich,2025-04-05,Switzerland,Super League,31,12,11,6,3,...,42.857,42.857,1.143,50.000,35.714,64.286,28.571,1.571,0.724747,1
65,Young Boys,Yverdon Sport,2025-04-05,Switzerland,Super League,31,3,1,10,10,...,66.667,33.333,2.133,13.333,0.000,46.667,40.000,1.000,0.939394,1


In [28]:
fl.create_import_file(all_predictions, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv", provider="lay_all_u25", market_name="Over/Under 2.5 Goals", selection_name="Under 2.5 Goals")

File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv
